In [16]:
from urllib.request import urlopen
import json as json
import pandas as pd
import numpy as np
import requests

## ETL USA

In [22]:
#importamos la informacion a traves de la API
url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&orderby=time"
df_usa= pd.read_csv(url)
pd.set_option('display.max_columns', None)

In [157]:
df_usa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11465 entries, 0 to 11464
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   time             11465 non-null  object 
 1   latitude         11465 non-null  float64
 2   longitude        11465 non-null  float64
 3   depth            11465 non-null  float64
 4   mag              11465 non-null  float64
 5   magType          11465 non-null  object 
 6   nst              8146 non-null   float64
 7   gap              8146 non-null   float64
 8   dmin             6049 non-null   float64
 9   rms              11465 non-null  float64
 10  net              11465 non-null  object 
 11  id               11465 non-null  object 
 12  updated          11465 non-null  object 
 13  place            11161 non-null  object 
 14  type             11465 non-null  object 
 15  horizontalError  7511 non-null   float64
 16  depthError       11465 non-null  float64
 17  magError    

In [23]:
def transformacion_usa(df):
    #elimino columnas innecesarias
    df= df.drop(columns=['magType', 'nst', 'gap', 'dmin', 'rms', 'net', 'id','updated','type', 'horizontalError', 'depthError', 'magError', 'magNst', 'status', 'locationSource', 'magSource'])
    #coloco todos los registros en minusculas
    df= df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    #Vamos a redondear las columnas de variables float
    df[['latitude','longitude','depth','mag']]=df[['latitude','longitude','depth','mag']].round(1)
    #eliminamos duplicados
    df= df.drop_duplicates()
    #agrego una columna "country" con el nombre del pais respectivo en caso que necesite identificar en procesos posteriores
    df['country']='usa'

    #cambiamos el formato de fecha para estandarizarla junto con los otros datasets
    df['time'] = pd.to_datetime(df['time'])
    df['time'] = df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    return df

In [24]:
df_usa= transformacion_usa(df_usa)

In [25]:
df_usa

,time,latitude,longitude,depth,mag,place,country
0,2023-05-09 20:24:58,38.8,-122.8,-0.5,1.1,"1km nne of the geysers, ca",usa
1,2023-05-09 20:08:46,19.2,-155.4,32.6,2.3,"11 km ene of p?hala, hawaii",usa
2,2023-05-09 20:03:52,34.5,-97.7,1.6,2.5,"9 km nw of velma, oklahoma",usa
3,2023-05-09 19:49:03,63.4,-147.7,75.5,1.3,"64 km e of cantwell, alaska",usa
4,2023-05-09 19:38:29,33.9,-117.5,1.6,1.1,"2km se of home gardens, ca",usa
...,...,...,...,...,...,...,...
11431,2023-04-10 17:54:29,55.1,-162.3,2.5,0.3,"3 km e of king cove, alaska",usa
11432,2023-04-10 17:50:05,58.2,-155.3,1.1,1.0,alaska peninsula,usa
11433,2023-04-10 17:46:50,58.2,-155.3,0.6,0.5,"85 km nw of karluk, alaska",usa
11434,2023-04-10 17:46:30,58.2,-155.3,1.3,1.3,"86 km nw of karluk, alaska",usa


## ETL Japon

In [57]:
#importamos los datos
url = "https://www.jma.go.jp/bosai/quake/data/list.json"
response = requests.get(url)
data = response.json()
df_japon = pd.DataFrame(data)

In [41]:
df_japon.head()

,ctt,eid,rdt,ttl,ift,ser,at,anm,acd,cod,mag,maxi,int,json,en_ttl,en_anm
0,20230511013145,20230511010819,2023-05-11T01:31:00+09:00,遠地地震に関する情報,発表,1,2023-05-11T01:02:00+09:00,南太平洋,950,-15.6-174.6-210000/,7.6,,[],20230511013145_20230511010819_VXSE5e_1.json,Distant Earthquake Information,South Pacific Ocean
1,20230510215952,20230510215351,2023-05-10T21:59:00+09:00,震源・震度情報,発表,1,2023-05-10T21:53:00+09:00,岩手県沿岸北部,210,+39.8+141.9-60000/,3.7,1,"[{'code': '02', 'maxi': '1', 'city': [{'code':...",20230510215952_20230510215351_VXSE5k_1.json,Earthquake and Seismic Intensity Information,Northern Coast of Iwate Prefecture
2,20230510215815,20230510215452,2023-05-10T21:58:00+09:00,震源・震度情報,発表,1,2023-05-10T21:54:00+09:00,能登半島沖,495,+37.6+137.3-10000/,5.1,3,"[{'code': '17', 'maxi': '3', 'city': [{'code':...",20230510215815_20230510215452_VXSE5k_1.json,Earthquake and Seismic Intensity Information,Off the Coast of Noto Peninsula
3,20230510215732,20230510215452,2023-05-10T21:57:00+09:00,震源に関する情報,発表,0,2023-05-10T21:54:00+09:00,能登半島沖,495,+37.6+137.3-10000/,5.1,,[],20230510215732_20230510215452_VXSE52_0.json,Earthquake Information,Off the Coast of Noto Peninsula
4,20230510215622,20230510215452,2023-05-10T21:56:00+09:00,震度速報,発表,0,2023-05-10T21:54:00+09:00,,,,,3,"[{'code': '17', 'maxi': '3', 'city': []}]",20230510215622_20230510215452_VXSE51_0.json,Seismic Intensity Information,


In [58]:
def transformacion_japon(df):
    #eliminamos las columnas que no vamos a utilizar
    df= df.drop(columns=['ctt','eid','rdt','ttl','ift','ser','anm','acd','maxi','int','json','en_ttl'])
    #eliminamos y/o reemplazamos caracateres innecesarios
    df['cod'] = df['cod'].str.replace('+', '', 1) # Esta linea de codigo reemplaza exclusivamente el primer '+'
    df['cod'] = df['cod'].str.replace('+', ',')
    df['cod'] = df['cod'].str.replace('-', ',')
    df['cod'] = df['cod'].str.replace('/', '')

    #ahora que esta limpio podemos separar los datos
    df = df.join(df['cod'].str.split(',', expand=True).rename(columns={0:'latitude', 1:'longitude', 2:'depth'}))
    df = df.drop(columns='cod')

    #convertimos a float la columna "depth"
    df['depth'] = df['depth'].astype('float64')
    #dividimos por mil para llevar la unidad de medida a KM para mantener la misma en todos los datasets
    df['depth'] = (df['depth'] / 1000)

    #renombramos columnas
    df = df.rename(columns={'at': 'time', 'en_anm': 'place'})
    #reordenamos columnas
    orden = ['time', 'latitude', 'longitude', 'depth', 'mag', 'place']
    df = df[orden]

    #todo a minusculas
    df=df.applymap(lambda x: x.lower() if isinstance(x,str) else x)
    #agrego una columna "country" con el nombre del pais respectivo en caso que necesite identificar en procesos posteriores
    df['country']='japon'

    #Vamos a redondear las columnas de variables float
    df[['latitude','longitude','depth','mag']]=df[['latitude','longitude','depth','mag']].round(1)

    #reemplazamos los "ｍ不明" ("desconocido en español") de la columna "mag" por NaN
    df['mag'] = df['mag'].replace({'ｍ不明': np.nan, '': np.nan})

    #eliminamos duplicados
    df= df.drop_duplicates()
    
    #substituimos los Nones por NaN
    df['longitude'] = df['longitude'].replace({None: np.nan, '': np.nan})
    df['latitude'] = df['latitude'].replace({None: np.nan, '': np.nan})

    #cambiamos el formato de fecha para estandarizarla junto con los otros datasets
    df['time'] = pd.to_datetime(df['time'])
    df['time'] = df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')

    #elimino los registos nulos pero solo aquellos que son nulos en todas las columnas
    df = df.dropna(subset=['latitude', 'longitude', 'depth', 'mag','place'], how='all')
    
    return df

In [59]:
df_japon= transformacion_japon(df_japon)

/var/folders/_5/spzfhw090m5dhqvxrw9qgsbr0000gn/T/ipykernel_3026/3376297249.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['cod'] = df['cod'].str.replace('+', '', 1) # Esta linea de codigo reemplaza exclusivamente el primer '+'
/var/folders/_5/spzfhw090m5dhqvxrw9qgsbr0000gn/T/ipykernel_3026/3376297249.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['cod'] = df['cod'].str.replace('+', ',')


In [60]:
df_japon

,time,latitude,longitude,depth,mag,place,country
0,2023-05-11 01:02:00,NaN,15.6,0.2,7.6,south pacific ocean,japon
1,2023-05-10 21:53:00,39.8,141.9,60.0,3.7,northern coast of iwate prefecture,japon
2,2023-05-10 21:54:00,37.6,137.3,10.0,5.1,off the coast of noto peninsula,japon
4,2023-05-10 21:54:00,NaN,NaN,NaN,NaN,,japon
5,2023-05-10 17:29:00,37.4,137.3,10.0,3.0,toyama bay,japon
...,...,...,...,...,...,...,...
252,2023-04-12 00:44:00,27.0,141.6,0.0,4.2,adjacent sea of​ chichijima island,japon
253,2023-04-12 00:38:00,42.6,143.8,100.0,4.3,off the coast of tokachi,japon
254,2023-04-11 09:09:00,27.0,142.7,60.0,4.5,adjacent sea of​ chichijima island,japon
255,2023-04-10 22:10:00,56.7,161.4,NaN,NaN,adjacent kamchatka peninsula,japon


## ETL Chile

In [54]:
#traemos la informacion a traves de la API
url = "https://api.xor.cl/sismo/recent"

response = requests.get(url)
data = response.json()
df_chile = pd.DataFrame(data)
df_chile

,status_code,status_description,events
0,0,Información obtenida satisfactoriamente,"{'id': '196662', 'url': 'http://sismologia.cl/..."
1,0,Información obtenida satisfactoriamente,"{'id': '196661', 'url': 'http://sismologia.cl/..."
2,0,Información obtenida satisfactoriamente,"{'id': '196659', 'url': 'http://sismologia.cl/..."
3,0,Información obtenida satisfactoriamente,"{'id': '196654', 'url': 'http://sismologia.cl/..."
4,0,Información obtenida satisfactoriamente,"{'id': '196643', 'url': 'http://sismologia.cl/..."
5,0,Información obtenida satisfactoriamente,"{'id': '196628', 'url': 'http://sismologia.cl/..."
6,0,Información obtenida satisfactoriamente,"{'id': '196627', 'url': 'http://sismologia.cl/..."
7,0,Información obtenida satisfactoriamente,"{'id': '196612', 'url': 'http://sismologia.cl/..."
8,0,Información obtenida satisfactoriamente,"{'id': '196609', 'url': 'http://sismologia.cl/..."
9,0,Información obtenida satisfactoriamente,"{'id': '196606', 'url': 'http://sismologia.cl/..."


In [47]:
#Veamos que caracteristicas tienen las columnas
df_chile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   status_code         30 non-null     int64 
 1   status_description  30 non-null     object
 2   events              30 non-null     object
dtypes: int64(1), object(2)
memory usage: 848.0+ bytes


In [53]:
def transformacion_chile(df):
    #Elminamos las columnas innecesarias y nos quedamos solo con la columna "events"
    df= df.drop(columns=['status_code','status_description'])

    #Extraemos la informacion y la adjuntamos al dataframe como nuevas columnas
    df['time']= df['events'].apply(lambda x : x['utc_date'])
    df['latitude']= df['events'].apply(lambda x : x['latitude'])
    df['longitude']=df['events'].apply(lambda x: x['longitude'])
    df['depth']=df['events'].apply(lambda x: x['depth'])
    df['mag']=df['events'].apply(lambda x: x['magnitude']['value'])
    df['place']=df['events'].apply(lambda x: x['geo_reference'])

    #Eliminamos la columna "events", ahora ya no la necesitamos
    df= df.drop(columns=['events'])

    #Pasamos todo a minusculas
    df= df.applymap(lambda x : x.lower() if isinstance(x,str) else x)
    #Agrego una columna "country" con el nombre del pais respectivo en caso que necesite identificar en procesos posteriores
    df['country']= 'chile'

    #convertimos el tipo de la columna "depth" a float para mantener la misma estructura en todos los datasets
    df['depth']= df['depth'].astype('Float64')
    #eliminamos duplicados
    df= df.drop_duplicates()
    #Vamos a redondear las columnas de variables float
    df[['latitude','longitude','depth','mag']]=df[['latitude','longitude','depth','mag']].round(1)

    #cambiamos el formato de fecha para estandarizarla junto con los otros datasets
    df['time'] = pd.to_datetime(df['time'])
    df['time'] = df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    return df

In [55]:
df_chile= transformacion_chile(df_chile)

In [56]:
df_chile

,time,latitude,longitude,depth,mag,place,country
0,2023-05-10 14:43:18,-30.9,-71.8,24.0,3.3,48 km al o de punitaqui,chile
1,2023-05-10 14:25:37,-30.8,-71.6,26.0,2.5,36 km al o de punitaqui,chile
2,2023-05-10 14:18:46,-21.7,-68.5,129.0,2.9,55 km al so de ollagüe,chile
3,2023-05-10 13:29:49,-23.3,-68.1,136.0,2.8,40 km al no de socaire,chile
4,2023-05-10 10:27:08,-28.5,-70.7,77.0,2.6,11 km al ne de vallenar,chile
5,2023-05-10 04:00:26,-21.1,-68.9,105.0,2.5,42 km al so de mina collahuasi,chile
6,2023-05-10 03:55:25,-31.8,-71.3,55.0,2.6,21 km al ne de los vilos,chile
7,2023-05-09 23:39:41,-26.1,-70.6,78.0,3.8,23 km al n de chañaral.,chile
8,2023-05-09 22:45:08,-21.2,-68.6,115.0,2.7,40 km al s de mina collahuasi,chile
9,2023-05-09 22:17:33,-24.8,-70.2,49.0,3.3,70 km al ne de taltal,chile


## ETL Final

In [ ]:
df_usa= transformacion_usa(df_usa)

In [179]:
df_final= pd.concat([df_usa,df_japon,df_chile])

In [180]:
df_final

,time,latitude,longitude,depth,mag,place,country
0,2023-05-09 19:49:03,63.4,-147.7,75.5,1.3,"64 km e of cantwell, alaska",usa
1,2023-05-09 19:38:29,33.9,-117.5,1.6,1.1,"2km se of home gardens, ca",usa
2,2023-05-09 19:36:10,38.8,-122.8,1.4,0.6,"10km nw of the geysers, ca",usa
3,2023-05-09 19:33:04,32.6,-117.0,3.0,1.4,"8km ese of chula vista, ca",usa
4,2023-05-09 19:25:28,38.8,-122.8,0.2,1.3,"8km wnw of cobb, ca",usa
...,...,...,...,...,...,...,...
20,2023-05-09 04:21:03,-21.2,-68.8,110.0,2.7,49 km al so de mina collahuasi,chile
21,2023-05-09 04:06:15,-23.4,-67.3,251.0,3.6,61 km al e de socaire,chile
22,2023-05-09 03:13:37,-21.1,-69.1,95.0,3.2,56 km al so de mina collahuasi,chile
23,2023-05-09 03:13:04,-35.0,-72.1,35.0,2.7,14 km al so de vichuquén,chile
